In [3]:
import yaml
from llama_index.llms.gemini import Gemini
from llama_index.core.query_engine import CitationQueryEngine
from llama_index.core import VectorStoreIndex, ServiceContext
from llama_index.readers.semanticscholar import SemanticScholarReader
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.core.agent import ReActAgent

c:\Users\acuca\anaconda3\envs\bibliographic_references_search\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Carregar o arquivo YAML
with open('../credentials.yaml', 'r') as file:
    credenciais = yaml.safe_load(file)
# Acessar as credenciais
gemini_api_key = credenciais['gemini_api_key']
serpapi_api_key = credenciais['serpapi_api_key']

In [5]:
llm=Gemini(model_name="models/gemini-1.0-pro-001", api_key=gemini_api_key)
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=GeminiEmbedding(model_name="models/embedding-001", api_key=gemini_api_key)
)

C:\Users\acuca\AppData\Local\Temp\ipykernel_7252\2871267505.py:2: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [6]:
from google_scholar_tool import GoogleScholarToolSpec
from main_theme_extractor_tool import ThemeExtractorTool

scholar_tool = GoogleScholarToolSpec(serpapi_api_key).to_tool_list()
query_key = ThemeExtractorTool(llm).to_tool_list()

tools = scholar_tool + query_key

In [7]:
context = """
You are responsible for summarizing the bibliographical references for a search. \n
You will be given a question and must use the tools available to find the central theme \n
of the search from the query and obtain papers related to that theme. 
\n\n
You should respond with:\n
* The answer to the question asked by the user\n
* And the references of all the articles found, including names, authors and access links
"""

In [8]:
agent = ReActAgent.from_tools(
    tools,
    llm=llm,
    verbose=True,
    context=context
)

In [9]:
response = agent.chat("What are the main techniques for segmenting cancer nodules in CT images?")
;print(str(response))

Thought: The current language of the user is: english. I need to use a tool to help me answer the question.
Action: extract_theme
Action Input: {'query': 'What are the main techniques for segmenting cancer nodules in CT images?'}
Observation: cancer nodule segmentation
Thought: I need to use a tool to help me answer the question.
Action: google_scholar_query
Action Input: {'query': 'cancer nodule segmentation', 'num_papers': 5}
Observation: [{'title': 'Review on lung nodule segmentation-based lung cancer classification using machine learning approaches', 'abstract': '… had significantly improved lung cancer classification due to better and precise lung and lung nodule segmentation. This improves the overall lung cancer classification with earlier …', 'authors_reviews_year_pub': 'S Shamas, SN Panda, I Sharma - Artificial Intelligence on Medical Data …, 2022 - Springer', 'link': 'https://link.springer.com/chapter/10.1007/978-981-19-0151-5_24'}, {'title': 'A bi-directional deep learning a

In [21]:
response.sources

[ToolOutput(content='cancer nodule segmentation', tool_name='extract_theme', raw_input={'args': (), 'kwargs': {'query': 'What are the main techniques for segmenting cancer nodules in CT images?'}}, raw_output='cancer nodule segmentation', is_error=False),
 ToolOutput(content="[{'title': 'Review on lung nodule segmentation-based lung cancer classification using machine learning approaches', 'abstract': '… had significantly improved lung cancer classification due to better and precise lung and lung nodule segmentation. This improves the overall lung cancer classification with earlier …', 'authors_reviews_year_pub': 'S Shamas, SN Panda, I Sharma - Artificial Intelligence on Medical Data …, 2022 - Springer', 'link': 'https://link.springer.com/chapter/10.1007/978-981-19-0151-5_24'}, {'title': 'A bi-directional deep learning architecture for lung nodule semantic segmentation', 'abstract': '… and efficient lung nodule segmentation method because they need to detect lung cancer at an … 1 expla

In [36]:
for tool in response.sources:
    if tool.tool_name == 'google_scholar_query':
        for ref in tool.raw_output:
            
            ref_str = f"{ref['title']}\n\n{ref['authors_reviews_year_pub']}\n\n{ref['link']}"
            print(ref_str)

Review on lung nodule segmentation-based lung cancer classification using machine learning approaches

S Shamas, SN Panda, I Sharma - Artificial Intelligence on Medical Data …, 2022 - Springer

https://link.springer.com/chapter/10.1007/978-981-19-0151-5_24
A bi-directional deep learning architecture for lung nodule semantic segmentation

D Bhattacharyya, N Thirupathi Rao, ESN Joshua… - The Visual …, 2023 - Springer

https://link.springer.com/article/10.1007/s00371-022-02657-1
Texture appearance model, a new model-based segmentation paradigm, application on the segmentation of lung nodule in the CT scan of the chest

F Shariaty, M Orooji, EN Velichko… - Computers in biology and …, 2022 - Elsevier

https://www.sciencedirect.com/science/article/pii/S0010482521008805
A hybrid deep learning model for effective segmentation and classification of lung nodules from CT images

M Murugesan, K Kaliannan, S Balraj… - Journal of Intelligent …, 2022 - content.iospress.com

https://content.iospress.c